In [1]:
import numpy as np
import pandas as pd
import os
import sys
sys.path.append(os.path.join(os.path.abspath('../'), 'predictions_collapsed'))
sys.path.append(os.path.join(os.path.abspath('../'), 'src'))
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
pd.set_option('display.max_colwidth', None)
import glob

from config_loader import (
    D_CONFIG, DATASET_TOP_PATH,
    DATASET_SITE_PATH, PROJECT_REPO_DIR, PROJECT_CONDA_ENV_YAML,
    DATASET_COLLAPSED_FEAT_DYNAMIC_INPUT_OUTPUT_PATH
    )

sys.path.append(os.path.join(PROJECT_REPO_DIR, 'src'))
from feature_transformation import *
from utils import load_data_dict_json

In [2]:
collapsed_feats_dir = '/home/prash/datasets/v20211018/HUF/split-by=patient_id/collapsed_features_dynamic_input_output/classifier_train_test_split'
collapsed_feats_file = os.path.join(collapsed_feats_dir, 'dynamic_featuresCustomTimes_10_6_labs_vitals.csv.gz')
collapsed_feats_df = pd.read_csv(collapsed_feats_file)

In [3]:
feats_per_tstep_dir = '/home/prash/datasets/v20211018/HUF/split-by=patient_id/features_per_timestep/classifier_train_test_split/'
feats_per_tstep_file = os.path.join(feats_per_tstep_dir, 'features_per_tstep_custom_times_labs_vitals.csv.gz')
outcomes_per_tstep_file = os.path.join(feats_per_tstep_dir, 'outcomes_per_tstep_custom_times_labs_vitals.csv.gz')
feats_per_tstep_df = pd.read_csv(feats_per_tstep_file)
outcomes_per_tstep_df = pd.read_csv(outcomes_per_tstep_file)

In [50]:
feats_per_tstep_df.shape

(1317461, 50)

In [6]:
outcomes_per_tstep_df['clinical_deterioration_outcome'].sum()/len(outcomes_per_tstep_df)

0.006139840192612912

In [26]:
feats_per_tstep_df['age_at_admission'] = feats_per_tstep_df.groupby(['patient_id','hospital_admission_id', 'facility_code'], sort=False)['age_at_admission'].apply(lambda x: x.ffill().bfill())


In [40]:
np.set_printoptions(precision=3, suppress=True)
pd.set_option('display.max_columns', None)
feats_per_tstep_df.iloc[:, 5:].describe().loc[['mean', '50%', 'std', 'max', 'count'], :]

,age_at_admission,alanine_aminotransferase,albumin_in_serum,alkaline_phosphatase,aspartate_aminotransferase,basophils,bicarbonate_venous_blood,bilirubin_in_serum,blood_glucose_concentration,bmi,body_temperature,calcium_in_serum,cholestrol_in_serum,creatinine_in_serum,diastolic_blood_pressure,eosinophils,erithrocytes,gamma_glutamyl_transferase,gender_is_male,gender_is_unknown,glucose_in_serum,heart_rate,height,hematocrit,hemoglobin,lactate_dehydrogenase,lymphocytes,monocytes,neutrophils,o2_sat,oxygen_venous_blood,ph_in_urine,ph_venous_blood,platelets,potassium_in_serum,prothrombin_time,respiratory_rate,reticulocytes_in_blood,sodium_in_serum,systolic_blood_pressure,triglyceride_in_serum,urate_in_serum,weight
mean,6.180598e+01,51.802136,3.231016,118.917346,61.093659,0.031824,25.143996,1.042697,165.654967,28.717565,36.322296,8.647230,156.558515,1.000052,69.128944,0.151233,3.953838,172.206167,0.543794,0.0,121.282571,79.388778,163.553684,54.293333,11.746909,252.944709,1.704463,0.760519,6.862739,95.552980,45.785040,5.878057,7.363148,50.033846,4.213034,8.98000,26.370781,56.710421,138.078428,124.487280,131.575814,5.542788,75.224414
50%,6.402466e+01,21.000000,3.300000,82.000000,29.000000,0.020000,24.900000,0.560000,148.000000,27.900000,36.300000,8.700000,153.000000,0.820000,68.500000,0.100000,3.960000,61.000000,1.000000,0.0,105.000000,79.000000,164.000000,57.400000,11.700000,201.000000,1.400000,0.670000,5.810000,96.000000,41.550000,5.500000,7.370000,16.900000,4.200000,10.10000,21.000000,49.300000,138.400000,122.500000,112.000000,5.200000,74.500000
std,2.085843e+01,181.087020,0.560410,139.132061,171.594296,0.090293,5.015996,2.054332,66.942240,6.679987,0.577976,0.719524,49.595551,0.696375,11.423885,0.260332,0.743632,353.369701,0.498079,0.0,52.966029,14.881837,9.314606,17.757924,2.238478,304.148246,3.248768,1.139285,4.656936,2.754274,21.602091,0.897586,0.064744,57.021970,0.580701,2.80749,13.247142,36.226328,4.181161,20.743614,103.346319,2.226515,19.524876
max,1.045781e+02,10725.000000,6.700000,3886.000000,11952.000000,15.560000,73.400000,47.510000,587.000000,60.000000,42.000000,16.000000,1248.500000,14.550000,170.000000,27.010000,8.510000,17464.000000,1.000000,0.0,1250.000000,190.000000,193.000000,89.400000,22.800000,20110.000000,180.850000,110.550000,171.500000,100.000000,281.000000,9.000000,7.730000,276.100000,11.700000,10.80000,60.000000,625.000000,186.600000,258.000000,4203.000000,27.200000,195.600000
count,1.317461e+06,67479.000000,61265.000000,49092.000000,28593.000000,113390.000000,23462.000000,66893.000000,212505.000000,3229.000000,603593.000000,58517.000000,32180.000000,108450.000000,611105.000000,113389.000000,113426.000000,44569.000000,839245.000000,839245.0,103137.000000,610975.000000,3298.000000,15.000000,80544.000000,31955.000000,113391.000000,113391.000000,113391.000000,386315.000000,23466.000000,16181.000000,23470.000000,130.000000,107302.000000,5.00000,316.000000,44629.000000,107320.000000,611162.000000,32173.000000,34510.000000,10851.000000


In [55]:
lab_features = parse_feature_cols(labs_data_dict)
feats_per_tstep_df.loc[:, lab_features].quantile([.5, .99, 1.0])

,alanine_aminotransferase,albumin_in_serum,alkaline_phosphatase,aspartate_aminotransferase,basophils,bicarbonate_venous_blood,bilirubin_in_serum,calcium_in_serum,cholestrol_in_serum,creatinine_in_serum,eosinophils,erithrocytes,gamma_glutamyl_transferase,glucose_in_serum,hematocrit,hemoglobin,lactate_dehydrogenase,lymphocytes,monocytes,neutrophils,oxygen_venous_blood,ph_in_urine,ph_venous_blood,platelets,potassium_in_serum,prothrombin_time,reticulocytes_in_blood,sodium_in_serum,triglyceride_in_serum,urate_in_serum
0.50,21.0,3.3,82.00,29.0,0.02,24.900,0.56,8.7,153.0,0.82,0.10,3.96,61.00,105.0,57.40,11.7,201.0,1.40,0.67,5.81,41.55,5.5,7.37,16.90,4.20,10.100,49.30,138.4,112.0,5.2
0.99,545.0,4.4,751.09,516.0,0.18,40.739,11.16,10.3,289.0,3.87,0.86,5.66,1631.64,325.0,86.95,17.1,1094.0,5.20,2.24,22.00,132.00,8.5,7.50,249.17,5.76,10.776,187.66,148.2,411.0,12.1
1.00,10725.0,6.7,3886.00,11952.0,15.56,73.400,47.51,16.0,1248.5,14.55,27.01,8.51,17464.00,1250.0,89.40,22.8,20110.0,180.85,110.55,171.50,281.00,9.0,7.73,276.10,11.70,10.800,625.00,186.6,4203.0,27.2


In [53]:
# simple outlier removal
new_feats_per_tstep_df = feats_per_tstep_df.copy()
outlier_cols_list=[]
for col in lab_features:
    col_median = np.nanpercentile(new_feats_per_tstep_df[col], 50)
    col_99_percentile = np.nanpercentile(new_feats_per_tstep_df[col], 99)
    col_max = np.nanpercentile(new_feats_per_tstep_df[col], 100)
    
    if (col_max-col_99_percentile)>2*(col_99_percentile-col_median):
        keep_inds = new_feats_per_tstep_df[col]<=np.nanpercentile(new_feats_per_tstep_df[col], 99)
        new_feats_per_tstep_df = new_feats_per_tstep_df[keep_inds]
new_feats_per_tstep_df

/home/prash/.conda/envs/madrid_data_prep/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1368: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(


,patient_id,hospital_admission_id,facility_code,start,stop,age_at_admission,alanine_aminotransferase,albumin_in_serum,alkaline_phosphatase,aspartate_aminotransferase,basophils,bicarbonate_venous_blood,bilirubin_in_serum,blood_glucose_concentration,bmi,body_temperature,calcium_in_serum,cholestrol_in_serum,creatinine_in_serum,diastolic_blood_pressure,eosinophils,erithrocytes,gamma_glutamyl_transferase,gender_is_male,gender_is_unknown,glucose_in_serum,heart_rate,height,hematocrit,hemoglobin,lactate_dehydrogenase,lymphocytes,monocytes,neutrophils,o2_sat,oxygen_venous_blood,ph_in_urine,ph_venous_blood,platelets,potassium_in_serum,prothrombin_time,respiratory_rate,reticulocytes_in_blood,sodium_in_serum,systolic_blood_pressure,triglyceride_in_serum,urate_in_serum,weight,admission_timestamp,stop_time
1570,1705,17518365,1,33.0,41.0,66.646576,28.0,3.6,84.0,28.0,0.00,NaN,NaN,151.0,NaN,36.5,9.1,239.0,1.16,80.0,0.00,4.38,71.0,1.0,0.0,167.0,84.0,NaN,NaN,NaN,379.0,1.80,0.70,9.40,96.0,NaN,6.0,NaN,NaN,4.10,NaN,NaN,44.2,137.0,140.0,286.0,10.70,NaN,2018-10-25 17:32:00,10:00:00
1585,1705,17518365,1,153.0,161.0,66.646576,22.0,3.2,60.0,22.0,0.00,NaN,0.38,105.0,NaN,36.5,9.0,201.0,0.82,82.0,0.00,4.56,62.0,1.0,0.0,96.0,83.0,NaN,NaN,NaN,383.0,2.20,0.60,8.20,NaN,NaN,NaN,NaN,NaN,4.10,NaN,NaN,67.9,141.0,179.0,243.0,6.50,NaN,2018-10-25 17:32:00,10:00:00
2319,11465,13048805,1,32.0,40.0,20.145205,17.0,3.5,41.0,30.0,0.01,29.7,0.36,NaN,NaN,NaN,8.8,158.0,1.05,55.0,0.06,2.83,22.0,1.0,0.0,83.0,62.0,NaN,NaN,NaN,121.0,1.19,0.21,1.54,NaN,35.0,NaN,NaN,NaN,3.52,NaN,NaN,35.9,137.3,82.0,106.0,5.20,NaN,2019-03-28 18:52:00,10:00:00
2405,11474,16747337,1,-9.0,-1.0,76.904110,10.0,3.6,117.0,13.0,0.01,NaN,0.38,NaN,NaN,NaN,9.2,176.0,1.51,NaN,0.01,4.54,153.0,0.0,0.0,161.0,NaN,NaN,NaN,NaN,229.0,0.66,0.36,12.31,NaN,NaN,NaN,NaN,NaN,5.11,NaN,NaN,41.8,140.4,NaN,55.0,9.30,NaN,2019-06-17 11:44:00,10:00:00
3874,12381,15198790,1,54.0,62.0,57.416440,16.0,4.0,150.0,19.0,0.00,NaN,0.28,190.0,NaN,36.0,9.8,259.0,1.01,76.0,0.20,4.85,48.0,1.0,0.0,213.0,76.0,NaN,NaN,NaN,141.0,2.20,0.50,6.00,NaN,NaN,NaN,NaN,NaN,4.40,NaN,NaN,NaN,136.0,136.0,288.0,9.89,NaN,2019-01-08 20:37:00,10:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1314056,1997394,14407186,1,45.0,53.0,42.273970,15.0,3.4,62.0,20.0,0.05,NaN,0.29,NaN,NaN,35.0,8.7,125.0,0.65,69.0,0.31,4.24,22.0,1.0,0.0,92.0,73.0,NaN,NaN,12.1,137.0,0.94,0.29,2.79,NaN,NaN,NaN,NaN,NaN,4.32,NaN,NaN,20.4,139.7,104.0,134.0,3.30,NaN,2020-11-10 13:25:00,18:00:00
1315684,1998408,13877645,1,87.0,95.0,67.873970,13.0,3.7,83.0,18.0,0.02,NaN,NaN,116.0,NaN,35.4,9.5,166.0,1.37,79.0,0.16,4.44,38.0,1.0,0.0,115.0,93.0,NaN,NaN,NaN,167.0,1.21,0.65,3.04,96.0,NaN,NaN,NaN,NaN,4.02,NaN,NaN,32.9,139.5,130.0,117.0,10.10,NaN,2019-08-04 19:18:00,18:00:00
1315980,1998669,19547134,1,-15.0,-7.0,50.687670,58.0,4.0,89.0,30.0,0.00,NaN,0.42,NaN,NaN,NaN,8.1,134.0,1.03,NaN,0.20,3.55,80.0,0.0,0.0,105.0,NaN,NaN,NaN,9.5,240.0,1.60,0.60,5.50,NaN,NaN,NaN,NaN,NaN,3.75,NaN,NaN,NaN,148.5,NaN,60.0,8.50,NaN,2020-11-30 17:05:00,10:00:00
1316802,1999471,17405401,1,194.0,202.0,59.109590,54.0,3.3,50.0,35.0,0.08,NaN,0.50,NaN,NaN,NaN,8.5,153.0,1.06,74.0,0.24,4.42,138.0,0.0,0.0,110.0,63.0,NaN,NaN,13.5,210.0,2.16,1.19,7.28,94.0,NaN,NaN,NaN,NaN,4.19,NaN,NaN,NaN,141.3,132.0,143.0,6.60,NaN,2023-02-03 00:09:00,10:00:00


In [21]:
from utils_preprocessing import get_preprocessed_data, get_all_features_data

In [22]:
preproc_data_dir = '/home/prash/datasets/v20211018/HUF/'
labs_df, labs_data_dict, vitals_df, vitals_data_dict, demographics_df, demographics_data_dict, medications_df, medications_data_dict, outcomes_df, outcomes_data_dict = get_preprocessed_data(preproc_data_dir)

features_df,features_data_dict = get_all_features_data(labs_df, labs_data_dict, 
                                                       vitals_df, vitals_data_dict, 
                                                       demographics_df, demographics_data_dict, 
                                                       medications_df, medications_data_dict, False)

In [25]:
features_df

,patient_id,hospital_admission_id,facility_code,hours_since_admission,alanine_aminotransferase,albumin_in_serum,alkaline_phosphatase,aspartate_aminotransferase,basophils,bicarbonate_venous_blood,...,sodium_in_serum,systolic_blood_pressure,triglyceride_in_serum,urate_in_serum,weight,birth_date,admission_timestamp,age_at_admission,gender_is_male,gender_is_unknown
0,121,11225357,1,-2.071,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,135.0,NaN,NaN,NaN,1994-06-29,2022-10-17 14:07:00,28.320548,1.0,0.0
1,121,11225357,1,26.895,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1994-06-29,2022-10-17 14:07:00,28.320548,1.0,0.0
2,121,11225357,1,27.181,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,120.0,NaN,NaN,NaN,1994-06-29,2022-10-17 14:07:00,28.320548,1.0,0.0
3,121,11225357,1,34.959,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1994-06-29,2022-10-17 14:07:00,28.320548,1.0,0.0
4,121,11225357,1,35.952,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,136.0,NaN,NaN,NaN,1994-06-29,2022-10-17 14:07:00,28.320548,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1793831,1999981,16226113,1,251.133,NaN,NaN,NaN,NaN,0.09,NaN,...,NaN,NaN,NaN,NaN,NaN,1987-02-15,2023-01-05 21:36:00,35.912329,0.0,0.0
1793832,1999981,16226113,1,251.400,NaN,3.7,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1987-02-15,2023-01-05 21:36:00,35.912329,0.0,0.0
1793833,1999981,16226113,1,251.500,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1987-02-15,2023-01-05 21:36:00,35.912329,0.0,0.0
1793834,1999981,16226113,1,251.967,NaN,NaN,NaN,NaN,NaN,NaN,...,139.3,NaN,NaN,NaN,NaN,1987-02-15,2023-01-05 21:36:00,35.912329,0.0,0.0


In [7]:
train_test_dir = '/home/prash/datasets/v20211018/HUF/split-by=patient_id/features_per_timestep/classifier_train_test_split/'
y_train_csv = os.path.join(train_test_dir, 'y_train_labs_vitals.csv.gz')
y_df = pd.read_csv(y_train_csv)
y_df

,patient_id,hospital_admission_id,facility_code,start,stop,stay_length,clinical_deterioration_outcome,admission_timestamp,stop_time
0,14338,19940756,1,-24.0,-22.0,62.85,0,2018-08-24 00:31:00,2:00:00
1,14338,19940756,1,-22.0,-14.0,62.85,0,2018-08-24 00:31:00,10:00:00
2,14338,19940756,1,-14.0,-6.0,62.85,0,2018-08-24 00:31:00,18:00:00
3,14338,19940756,1,-6.0,2.0,62.85,0,2018-08-24 00:31:00,2:00:00
4,14338,19940756,1,2.0,10.0,62.85,0,2018-08-24 00:31:00,10:00:00
...,...,...,...,...,...,...,...,...,...
802126,153128,18872581,1,26.0,34.0,65.70,0,2021-08-23 00:14:00,10:00:00
802127,153128,18872581,1,34.0,42.0,65.70,0,2021-08-23 00:14:00,18:00:00
802128,153128,18872581,1,42.0,50.0,65.70,0,2021-08-23 00:14:00,2:00:00
802129,153128,18872581,1,50.0,58.0,65.70,0,2021-08-23 00:14:00,10:00:00


In [8]:
y_df['clinical_deterioration_outcome'].sum()/len(y_df)

0.006284509637453234

In [9]:
len(y_df['hospital_admission_id'].unique())

39732